In [2]:
from sklearn.cluster import OPTICS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def convet_to_float_array(input: str) -> np.array:
    values_str = input.strip('[]')
    values_list = values_str.split(',')
    # 将字符串列表转换为浮点数数组
    float_array = np.array([float(value) for value in values_list])
    return float_array

# 读取title embedding
title_embedding = np.array(pd.read_csv("../data/azure_title_test_2_embedding.csv", encoding='utf-8')["title_embedding"].apply(lambda x: convet_to_float_array(x)).tolist())

from sklearn.metrics.pairwise import pairwise_distances
# 计算相似度矩阵
similarity_matrix = 1 - pairwise_distances(title_embedding, metric='cosine')


from sklearn.cluster import OPTICS

# 创建OPTICS对象并拟合相似度矩阵
optics = OPTICS(min_samples=5, xi=0.002, min_cluster_size=5)
clusters = optics.fit_predict(similarity_matrix)

# print(clusters)

/Users/huqianghui/Downloads/1.futu-openAI/event_creation/.venv/lib/python3.10/site-packages/sklearn/cluster/_optics.py:903: RuntimeWarning: divide by zero encountered in divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


In [3]:
import numpy as np
# 将ndarray对象保存到文件中
np.save('clusters-002-0505.npy', clusters)

In [6]:
import numpy as np
# 将ndarray对象保存到文件中
clusters = np.load('clusters-002-0505.npy')

In [7]:
# 对cluster进行分类，以元素值为key，index List作为value的字典
index_cluster_map = {}
for i, cluster_id in enumerate(clusters):
    # -1的元素作为噪声点，丢弃
    if cluster_id != -1:
        if cluster_id not in index_cluster_map:
            index_cluster_map[cluster_id] = []
        index_cluster_map[cluster_id].append(i)

In [8]:
import pandas as pd

source_title_df = pd.read_csv("../data/azure_title_test_2_embedding.csv")
# 根据cluster_map 里面内容，转化为对应event title的map
title_cluster_map = {}

# 将索引列表转换为标题列表
for cluster_id, index_list in index_cluster_map.items():
    title_list = source_title_df.iloc[index_list]['title'].tolist()
    title_cluster_map[cluster_id] = title_list

In [9]:
title_list_str_map = {}
# 将标题列表拼接成一个字符串
for cluster_id, title_list in title_cluster_map.items():
    title_str = '\n'.join([f'{i+1}. {title}' for i, title in enumerate(title_list)])
    title_list_str_map[cluster_id] = title_str

In [28]:
import sys
sys.path.append("..")
import openAIRoundRobin.eventCreationFromTitle as eventCreationFromTitle

# from importlib import reload # reload 
# reload(eventCreationFromTitle)

event_map={}
# 遍历cluster_map字典，通过gpt4生成event
for cluster_id, title_str in title_list_str_map.items():
    eventFinal = eventCreationFromTitle.gpt4ProcessByEventCreationTemplate(title_str)
    event_map[cluster_id]= eventFinal

openai index: 0
Q2 earnings and revenue rise for multiple companies; several raise 2023 guidance.
openai index: 1
珀莱雅上半年净利润增65%，Uber二季度营业利润转正，小米收入跌7%，瑞幸净收入增88%，AMD净利润暴跌94%。
openai index: 2
Japanese companies report mixed 1Q financial results.
openai index: 3
Mini LED业务深化，动力电池回收商业化，汽车电子市场拓展，超导技术关注增加。
openai index: 4
重庆稳就业扩岗位，蔚来ES8起火事故，企业捐款抗汛救灾，交通运输防汛抢险。
openai index: 5
Mining companies progress in drilling, exploration, and approvals; shares rise.
openai index: 0
Mining companies explore and produce lithium, copper, and gold in various projects.
openai index: 1
2023年7月31日，多家上市公司公布股东人数及高管减持情况。
openai index: 2
8月2日多家基金重仓股涨跌互现，涉及科技、医药、房地产等行业。
openai index: 3
多家公司高管变动；上海迪士尼与滴露战略联盟；梅西代言中国白酒。
openai index: 4
Changyang Tech focuses on R&D, supplies Mini LED to Samsung, LG; Guoxin Tech develops automotive chips, partners with robot companies; Zeyu Intelligence expands in power grid business; Huafeng Tech targets NEV connectors; Kirin Xinan improves storage security products; Shihua Tech expands

In [45]:
import sys
sys.path.append("..")
import openAIRoundRobin.eventCreationFromTitle as eventCreationFromTitle

from importlib import reload # reload 
reload(eventCreationFromTitle)

filtered_event_map={}
# 遍历cluster_map字典，通过gpt4生成event
for cluster_id, event_str in event_map.items():
    filteredResulst = eventCreationFromTitle.gpt4ProcessByEventFilterTemplate(event_str)
    if "yes" in filteredResulst.lower():
        filtered_event_map[cluster_id] = event_str
        print(event_str) 

openai index: 0
Yes.
Q2 earnings and revenue rise for multiple companies; several raise 2023 guidance.
openai index: 1
yes.
珀莱雅上半年净利润增65%，Uber二季度营业利润转正，小米收入跌7%，瑞幸净收入增88%，AMD净利润暴跌94%。
openai index: 2
Yes.
Japanese companies report mixed 1Q financial results.
openai index: 3
yes.
Mini LED业务深化，动力电池回收商业化，汽车电子市场拓展，超导技术关注增加。
openai index: 4
yes.
重庆稳就业扩岗位，蔚来ES8起火事故，企业捐款抗汛救灾，交通运输防汛抢险。
openai index: 5
Yes.
Mining companies progress in drilling, exploration, and approvals; shares rise.
openai index: 0
Yes.
Mining companies explore and produce lithium, copper, and gold in various projects.
openai index: 1
yes.
2023年7月31日，多家上市公司公布股东人数及高管减持情况。
openai index: 2
no.
openai index: 3
yes.
多家公司高管变动；上海迪士尼与滴露战略联盟；梅西代言中国白酒。
openai index: 4
yes.
Changyang Tech focuses on R&D, supplies Mini LED to Samsung, LG; Guoxin Tech develops automotive chips, partners with robot companies; Zeyu Intelligence expands in power grid business; Huafeng Tech targets NEV connectors; Kirin Xinan improves storage security product

In [46]:
print(filtered_event_map)

{27: 'Q2 earnings and revenue rise for multiple companies; several raise 2023 guidance.', 11: '珀莱雅上半年净利润增65%，Uber二季度营业利润转正，小米收入跌7%，瑞幸净收入增88%，AMD净利润暴跌94%。', 34: 'Japanese companies report mixed 1Q financial results.', 1: 'Mini LED业务深化，动力电池回收商业化，汽车电子市场拓展，超导技术关注增加。', 17: '重庆稳就业扩岗位，蔚来ES8起火事故，企业捐款抗汛救灾，交通运输防汛抢险。', 54: 'Mining companies progress in drilling, exploration, and approvals; shares rise.', 53: 'Mining companies explore and produce lithium, copper, and gold in various projects.', 2: '2023年7月31日，多家上市公司公布股东人数及高管减持情况。', 15: '多家公司高管变动；上海迪士尼与滴露战略联盟；梅西代言中国白酒。', 16: 'Changyang Tech focuses on R&D, supplies Mini LED to Samsung, LG; Guoxin Tech develops automotive chips, partners with robot companies; Zeyu Intelligence expands in power grid business; Huafeng Tech targets NEV connectors; Kirin Xinan improves storage security products; Shihua Tech expands optoelectronic materials business; Lihewei explores smart grids; Yuntian Lifei sells AI chips abroad, develops robot platforms; Junpu Intell

In [48]:
cluster_records_df_map ={}
filter_cluster_records_df_map ={}

# 把index list的值，转化为csv满足内容的dataframe
for cluster_id, index_list in index_cluster_map.items():
    record_cluster_df = pd.DataFrame(columns=source_title_df.columns)
    record_cluster_df = source_title_df.iloc[index_list]
    cluster_records_df_map[cluster_id] = record_cluster_df

# 把每个clustered dataframe插入event列，同时第一行赋值为相应的gpt4生成的event内容
for cluster_id, record_cluster_df in cluster_records_df_map.items():
    if cluster_id in filtered_event_map:
        first_index = record_cluster_df.index[0]
        col_event = pd.Series(filtered_event_map[cluster_id])        
        record_cluster_df.insert(0, 'event',col_event)
        record_cluster_df.loc[int(first_index),"event"] = filtered_event_map[cluster_id]
        filter_cluster_records_df_map[cluster_id] = record_cluster_df

# 将值列表合并到一个大列表中
merged_cluster_records_df = pd.concat(filter_cluster_records_df_map.values(), axis=0)

#print(merged_cluster_records_df)

# 保存到csv文件中
#merged_cluster_records_df.to_csv('final_event_result_v2.csv', index=False)
merged_cluster_records_df.to_csv('final_event_result_4excel_v2.csv',encoding='utf-8-sig',index=False)